In [1]:
import numpy as np
import pandas as pd
import os
import glob
import mne

# from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

# Analysis Plan
## Experiments 
1. Left vs. Right
2. Inward vs. Upward vs. Downward
3. Open and close vs. Rest
4. Open vs. Close
    1. ROIs
    2. Contralteral and Ipsilateral classification
    3. Non-ROI classification

## Features
- [x] Power Spectral Density
- [x] Spectopo Plots
- [x] Time-series
- [x] ICA Activations
- [ ] ERS and ERD

## Algorithms
1. ML
    1. PSD [band-wise and entire-band]
    2. Time-series
2. CNN
    1. spectopo
3. LSTM
    1. time-series


# 1. Frequency Domain [PSD]

In [2]:
dataset = pd.read_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/cleaned_data_asr.csv")
dataset

,0,1,2,3,4,5,6,7,8,9,...,1096,1097,1098,1099,handPos,Hand,Action,Subject,Channel,Trial
0,-4.671667,-4.451136,-4.263363,-4.122293,-4.039534,-4.023682,-4.079801,-4.209121,-4.408905,-4.672575,...,-8.788404,-8.901701,-8.911717,-8.817698,palmDown,Right,Open,P01,1,1
1,6.391617,6.335880,6.244392,6.109434,5.925195,5.688357,5.398465,5.058200,4.673382,4.252845,...,-20.972265,-20.743320,-20.478785,-20.176245,palmDown,Right,Open,P01,2,1
2,0.160325,0.081283,-0.010762,-0.123249,-0.262893,-0.435155,-0.643720,-0.890082,-1.173304,-1.489870,...,7.087850,7.448699,7.722362,7.906469,palmDown,Right,Open,P01,3,1
3,-1.336037,-1.659968,-1.983633,-2.302723,-2.612850,-2.909530,-3.188195,-3.444230,-3.673032,-3.870121,...,10.234831,10.642152,11.007372,11.328000,palmDown,Right,Open,P01,4,1
4,12.797042,12.577418,12.352886,12.123366,11.889187,11.651197,11.410817,11.170057,10.931464,10.698002,...,4.104468,3.970622,3.816026,3.642702,palmDown,Right,Open,P01,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384955,-1.525676,-1.255532,-0.849529,-0.324107,0.302524,1.010990,1.781305,2.593337,3.427199,4.263574,...,-9.633747,-10.102177,-10.289121,-10.183593,palmUp,Right,Close,FP12,58,480
384956,0.430836,1.836212,3.301004,4.780484,6.231448,7.613512,8.890228,10.029982,11.006654,11.800056,...,-18.231979,-18.785727,-18.982012,-18.810005,palmUp,Right,Close,FP12,59,480
384957,2.463603,3.943370,5.494955,7.070804,8.621435,10.097134,11.449710,12.634233,13.610693,14.345462,...,-9.042601,-8.470536,-7.647579,-6.599504,palmUp,Right,Close,FP12,60,480
384958,-4.482436,-4.953345,-5.120734,-4.995581,-4.599613,-3.963956,-3.127460,-2.134715,-1.033886,0.125495,...,-4.829801,-5.120802,-5.356338,-5.527806,palmUp,Right,Close,FP12,61,480


In [3]:
# Find list of channels available for every participant
chans, chan_count = np.unique(dataset["Channel"], return_counts=True)
good_chans = [i+1 for i in range(len(chan_count)) if chan_count[i] == 6240]
print(len(good_chans))
print(good_chans)

48
[2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 38, 40, 42, 43, 44, 45, 46, 47, 48, 50, 53, 54, 55, 56, 57, 58, 59, 60, 61]


In [4]:
# Subject-wise channel availability
sub_good_chans = {}
for sub in np.unique(dataset["Subject"]):
    sub_good_chans[sub] = np.unique(dataset[dataset["Subject"] == sub]["Channel"]) 
sub_good_chans

{'FP02': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 'FP03': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54,
        55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 'FP05': array([ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
        53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]),
 'FP06': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 22, 23, 24, 25, 26

In [5]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/ASR")
psd_data = pd.read_csv("psd_features_v1.csv", header=None, names=list(range(1, 385)))
psd_oc = np.load("psd_oc_labels.npy")
psd_lr = np.load("psd_lr_labels.npy")
psd_pos = np.load("psd_handpos_labels.npy")
psd_chan = np.load("psd_chans_labels.npy", allow_pickle=True)
psd_sub = np.load("psd_subjs_labels.npy", allow_pickle=True)

psd_data.shape,

((6240, 384),)

In [6]:
psd_data

,1,2,3,4,5,6,7,8,9,10,...,375,376,377,378,379,380,381,382,383,384
0,261.365626,84.885405,9.326743,34.385464,12.999225,59.011496,105.911403,58.178897,12.122176,59.735714,...,20.579816,27.147481,26.248709,72.846230,144.925389,56.174803,17.087038,23.993080,15.787540,56.180855
1,159.397705,25.611828,28.509104,29.566041,32.887987,93.283418,129.471277,35.842015,11.123689,31.927731,...,22.756594,12.752659,7.760852,46.004937,121.749275,28.232201,14.292694,10.957113,7.552433,34.694225
2,42.739529,20.334689,3.284019,16.178111,21.206371,39.513974,8.503900,5.929776,7.909893,22.947588,...,13.951308,13.471479,6.647135,35.444086,12.007450,10.196942,15.237750,10.347288,5.778238,32.494474
3,43.264261,11.935379,8.510044,17.471082,12.883778,38.821159,8.070000,6.456800,7.455867,40.775420,...,7.458639,10.768673,8.911387,27.578205,26.209023,6.017779,7.251091,7.966774,7.255972,22.849738
4,82.365826,15.514912,9.922482,15.716173,5.674711,32.230197,7.970801,4.919606,6.731930,44.149726,...,17.602682,18.756810,6.990275,46.498109,37.979964,15.161350,16.238745,17.156454,5.662176,41.650303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6235,440.326764,145.808163,34.475225,40.260243,16.053948,91.573274,159.180016,48.399535,10.496874,8.447043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6236,692.190534,381.081860,28.715601,28.137909,6.982059,64.005111,127.655648,75.081057,6.923525,2.768938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6237,317.384655,110.995845,21.890656,41.004525,12.949552,76.182781,39.234880,22.450794,5.084134,9.399115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6238,60.883400,14.369868,5.744175,17.524368,8.152813,33.256499,12.846058,3.644452,1.475327,3.935848,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
req_chan_idx = {}
for key in sub_good_chans.keys():
    req_chan_idx[key] = [i for i in range(len(sub_good_chans[key])) if sub_good_chans[key][i] in good_chans] 
print(req_chan_idx)

{'FP02': [1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 39, 41, 42, 43, 44, 45, 46, 47, 49, 52, 53, 54, 55, 56, 57, 58, 59, 60], 'FP03': [1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 39, 40, 41, 42, 43, 44, 45, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57], 'FP05': [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 36, 38, 40, 41, 42, 43, 44, 45, 46, 48, 51, 52, 53, 54, 55, 56, 57, 58, 59], 'FP06': [1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 36, 38, 40, 41, 42, 43, 44, 45, 46, 48, 51, 52, 53, 54, 55, 56, 57, 58, 59], 'FP07': [1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 39, 41, 42, 43, 44, 45, 46, 47, 49, 52, 53, 54, 55, 56, 57, 58, 59, 60], 'FP08': [1, 2, 3, 4, 5, 7

In [8]:
# Extract PSD data for common good channels
subjects = list(req_chan_idx.keys())
ntrials = 480
eeg_data = np.array(())
for i in range(len(subjects)):
    
    # Selecting rows
    row_start = i * ntrials
    row_end = (i+1) * ntrials
    
    # Selecting columns based on good channels
    p_eeg = np.array(())
    cols = req_chan_idx[subjects[i]]
    for ii in cols:
        col_start = (ii) * 6 
        col_end = (ii+1) * 6
        
        if p_eeg.shape[0] != 0:
            p_eeg = np.concatenate((p_eeg, psd_data.iloc[row_start: row_end, col_start: col_end].to_numpy()), axis=1)
        else:
            p_eeg = psd_data.iloc[row_start: row_end, col_start: col_end].to_numpy()    
    
    eeg_data = p_eeg if eeg_data.shape[0] == 0 else np.concatenate((eeg_data, p_eeg))

eeg_data_df = pd.DataFrame(np.array(eeg_data))
eeg_data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,278,279,280,281,282,283,284,285,286,287
0,105.911403,58.178897,12.122176,59.735714,21.831066,95.386830,59.877859,40.133826,13.297788,14.011689,...,6.954534,19.233276,5.306742,32.800473,40.176999,29.638491,10.017080,30.270492,9.295995,51.887830
1,129.471277,35.842015,11.123689,31.927731,12.178638,58.171418,73.815901,24.971421,11.119770,19.750610,...,10.460669,19.411593,7.710000,39.352713,52.031180,32.509928,30.594962,20.009758,12.132955,67.174055
2,8.503900,5.929776,7.909893,22.947588,14.487927,46.540607,10.131950,5.368680,7.521590,19.696717,...,6.468891,15.357873,4.979424,28.372416,23.281347,11.840894,5.782716,9.545962,8.577419,23.705579
3,8.070000,6.456800,7.455867,40.775420,14.996896,64.151913,8.656914,5.568085,7.917807,15.336780,...,6.271563,14.338439,8.244447,29.773503,28.881036,9.812493,7.465496,7.841785,9.909947,25.561386
4,7.970801,4.919606,6.731930,44.149726,18.400837,69.176610,4.584238,4.433566,6.804257,17.360087,...,4.809282,11.830967,5.446180,23.216197,31.868529,12.680218,11.662428,13.391449,7.036540,33.116223


In [9]:
X = eeg_data_df.to_numpy()
y = psd_lr

# y = psd_pos
# y = psd_oc
X.shape, y.shape

((6240, 288), (6240,))

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Define the parameter grids for each classifier
xgb_param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.1, 0.05, 0.01]
}

lgbm_param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.1, 0.05, 0.01]
}

svc_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

# Define the classifiers
xgb_clf = XGBClassifier()
lgbm_clf = LGBMClassifier()
svc_clf = SVC()

# Create a dictionary of classifiers and their corresponding parameter grids
classifiers = {
    'XGBoost': (xgb_clf, xgb_param_grid),
    'LightGBM': (lgbm_clf, lgbm_param_grid),
    'SVC': (svc_clf, svc_param_grid)
}

grid_scores_list = []
# Perform grid search cross-validation for each classifier
for clf_name, (clf, param_grid) in classifiers.items():
    grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_red, y)
    
    print(f"Results for {clf_name}:")
    print("Best parameters:", grid_search.best_params_)
    print("Best score:", grid_search.best_score_)
    print()
    grid_scores_list.append(grid_search.cv_results_)


XGBoostError: 
XGBoost Library (libxgboost.so) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["/home/shivam/anaconda3/lib/python3.9/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /home/shivam/anaconda3/lib/libxgboost.so)"]


In [ ]:
import pickle
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/GridSearchCV")
with open("gscv-psd-left-right.pkl", "wb") as f:
    pickle.dump(grid_scores_list, f)

## 6 bands

### 1. Left vs. Right [6 band]

In [43]:
X_train, X_eval, y_train, y_eval = train_test_split(eeg_data_df, psd_lr, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape#, y_test.shape

((4992, 288), (1248, 288), (4992,), (1248,))

In [44]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [00:48<00:00,  1.67s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.83,0.83,0.83,0.83,3.61
RandomForestClassifier,0.82,0.82,0.82,0.82,4.49
LGBMClassifier,0.82,0.82,0.82,0.82,1.21
SVC,0.81,0.81,0.81,0.81,3.68
NuSVC,0.80,0.80,0.80,0.80,3.90
CalibratedClassifierCV,0.80,0.80,0.80,0.80,9.68
ExtraTreesClassifier,0.80,0.80,0.80,0.80,0.99
LinearSVC,0.80,0.80,0.80,0.80,2.68
BaggingClassifier,0.79,0.79,0.79,0.79,7.93


In [45]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/psd-left-vs-right-all.csv")

### 2. Hand positions [6 band]

In [46]:
X_train, X_eval, y_train, y_eval = train_test_split(eeg_data_df, psd_pos, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4992, 288), (1248, 288), (4992,), (1248,))

In [47]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [01:22<00:00,  2.84s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.87,0.87,None,0.87,7.70
LGBMClassifier,0.86,0.86,None,0.86,2.06
RandomForestClassifier,0.81,0.81,None,0.81,5.11
ExtraTreesClassifier,0.80,0.80,None,0.80,1.12
BaggingClassifier,0.78,0.78,None,0.78,8.42
NuSVC,0.74,0.74,None,0.74,5.31
KNeighborsClassifier,0.69,0.69,None,0.69,0.10
SVC,0.68,0.68,None,0.68,4.54
DecisionTreeClassifier,0.65,0.65,None,0.65,1.34


In [48]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/psd-in-vs-up-vs-down-all.csv")

### 3. Open vs. Close [6 band]

In [49]:
X_train, X_eval, y_train, y_eval = train_test_split(eeg_data_df, psd_oc, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4992, 288), (1248, 288), (4992,), (1248,))

In [50]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [00:55<00:00,  1.92s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NuSVC,0.56,0.56,0.56,0.56,5.95
RandomForestClassifier,0.56,0.56,0.56,0.56,5.82
LGBMClassifier,0.55,0.55,0.55,0.55,0.62
LogisticRegression,0.55,0.55,0.55,0.55,0.12
LinearDiscriminantAnalysis,0.55,0.55,0.55,0.55,0.25
RidgeClassifierCV,0.55,0.55,0.55,0.55,0.11
RidgeClassifier,0.55,0.55,0.55,0.55,0.08
LinearSVC,0.54,0.54,0.54,0.54,3.30
SVC,0.54,0.54,0.54,0.54,4.38


In [51]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/psd-open-vs-close-all.csv")

## 2. Entire frequency band

In [52]:
eeg_data_all_df = eeg_data_df.iloc[:, ::-6].sort_index(axis=1)
eeg_data_all_df

,5,11,17,23,29,35,41,47,53,59,...,233,239,245,251,257,263,269,275,281,287
0,95.39,34.22,5.35,175.52,15.72,37.32,23.85,59.09,30.64,43.27,...,54.13,28.76,13.65,41.99,57.63,26.93,39.28,27.02,32.80,51.89
1,58.17,37.53,7.98,170.06,16.73,42.92,24.89,72.92,33.95,56.81,...,41.05,21.67,13.17,57.84,62.72,25.90,52.84,41.68,39.35,67.17
2,46.54,36.19,5.86,130.36,18.13,36.16,28.91,82.26,32.43,51.05,...,50.71,32.26,14.34,22.90,50.38,25.10,32.12,28.15,28.37,23.71
3,64.15,30.25,4.54,172.87,14.52,26.28,21.82,47.68,39.09,58.45,...,24.36,14.47,7.70,27.02,49.11,16.69,33.41,30.17,29.77,25.56
4,69.18,31.91,7.55,144.77,16.25,27.93,32.94,62.69,47.32,64.05,...,53.32,26.03,16.47,24.51,54.43,17.48,24.52,22.05,23.22,33.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6235,91.57,21.74,3.81,336.91,33.04,96.07,34.85,77.83,20.64,19.44,...,5.27,3.52,6.57,45.07,9.22,8.00,17.33,15.78,24.43,112.43
6236,64.01,10.70,2.52,171.88,12.99,40.51,11.33,25.59,6.14,4.40,...,3.40,2.24,2.86,14.40,6.13,7.16,7.66,22.27,22.14,26.90
6237,76.18,16.06,3.07,219.93,16.02,41.35,11.18,25.25,7.60,8.36,...,2.79,3.03,3.94,22.22,12.17,14.02,27.90,26.26,24.04,54.98
6238,33.26,7.59,2.87,46.36,6.33,8.23,7.93,12.22,6.88,8.18,...,2.63,2.83,2.34,22.55,20.32,11.27,35.21,39.37,15.00,54.01


In [53]:
entire_band_idx = list(eeg_data_all_df.columns)
print(entire_band_idx)

[5, 11, 17, 23, 29, 35, 41, 47, 53, 59, 65, 71, 77, 83, 89, 95, 101, 107, 113, 119, 125, 131, 137, 143, 149, 155, 161, 167, 173, 179, 185, 191, 197, 203, 209, 215, 221, 227, 233, 239, 245, 251, 257, 263, 269, 275, 281, 287]


### 1. Left vs. Right

In [73]:
X_train, X_eval, y_train, y_eval = train_test_split(eeg_data_all_df.to_numpy(), psd_lr, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4992, 48), (1248, 48), (4992,), (1248,))

In [74]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [00:13<00:00,  2.10it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NuSVC,0.80,0.80,0.80,0.80,1.64
LGBMClassifier,0.78,0.78,0.78,0.78,0.42
SVC,0.78,0.78,0.78,0.78,1.40
XGBClassifier,0.78,0.78,0.78,0.78,0.72
RandomForestClassifier,0.77,0.77,0.77,0.77,1.94
ExtraTreesClassifier,0.77,0.77,0.77,0.77,0.53
CalibratedClassifierCV,0.77,0.76,0.76,0.77,2.46
LogisticRegression,0.77,0.76,0.76,0.77,0.06
LinearSVC,0.77,0.76,0.76,0.77,0.63


In [75]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/psd-left-vs-right-single.csv")

### 2. Hand positions

In [57]:
X_train, X_eval, y_train, y_eval = train_test_split(eeg_data_all_df.to_numpy(), psd_pos, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4992, 48), (1248, 48), (4992,), (1248,))

In [58]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [00:23<00:00,  1.23it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.77,0.77,None,0.77,0.46
XGBClassifier,0.77,0.77,None,0.77,2.28
ExtraTreesClassifier,0.75,0.75,None,0.75,0.56
RandomForestClassifier,0.75,0.75,None,0.75,2.03
NuSVC,0.72,0.72,None,0.72,2.33
BaggingClassifier,0.71,0.71,None,0.71,1.45
KNeighborsClassifier,0.69,0.69,None,0.69,0.05
LabelSpreading,0.67,0.67,None,0.67,0.52
LabelPropagation,0.67,0.67,None,0.67,0.39


In [59]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/psd-in-vs-up-vs-down-single.csv")

### 3. Open vs. Close

In [60]:
X_train, X_eval, y_train, y_eval = train_test_split(eeg_data_all_df.to_numpy(), psd_oc, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4992, 48), (1248, 48), (4992,), (1248,))

In [61]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [00:18<00:00,  1.57it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.54,0.54,0.54,0.54,0.70
NuSVC,0.53,0.53,0.53,0.53,5.04
NearestCentroid,0.53,0.53,0.53,0.52,0.01
XGBClassifier,0.52,0.52,0.52,0.52,0.57
BaggingClassifier,0.53,0.52,0.52,0.52,1.74
ExtraTreeClassifier,0.52,0.52,0.52,0.52,0.02
LGBMClassifier,0.52,0.52,0.52,0.52,0.18
RidgeClassifier,0.52,0.52,0.52,0.52,0.02
GaussianNB,0.51,0.52,0.52,0.44,0.01


In [62]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/psd-open-vs-close-single.csv")

## 3. Five frequency bands

In [63]:
eeg_data_five_df = eeg_data_df.loc[:, ~eeg_data_df.columns.isin(entire_band_idx)]
eeg_data_five_df

,0,1,2,3,4,6,7,8,9,10,...,276,277,278,279,280,282,283,284,285,286
0,105.91,58.18,12.12,59.74,21.83,59.88,40.13,13.30,14.01,5.43,...,35.96,20.13,6.95,19.23,5.31,40.18,29.64,10.02,30.27,9.30
1,129.47,35.84,11.12,31.93,12.18,73.82,24.97,11.12,19.75,4.21,...,30.04,9.57,10.46,19.41,7.71,52.03,32.51,30.59,20.01,12.13
2,8.50,5.93,7.91,22.95,14.49,10.13,5.37,7.52,19.70,7.14,...,6.05,3.99,6.47,15.36,4.98,23.28,11.84,5.78,9.55,8.58
3,8.07,6.46,7.46,40.78,15.00,8.66,5.57,7.92,15.34,5.25,...,10.18,4.78,6.27,14.34,8.24,28.88,9.81,7.47,7.84,9.91
4,7.97,4.92,6.73,44.15,18.40,4.58,4.43,6.80,17.36,5.65,...,9.29,3.76,4.81,11.83,5.45,31.87,12.68,11.66,13.39,7.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6235,440.33,145.81,34.48,40.26,16.05,159.18,48.40,10.50,8.45,1.96,...,70.29,25.12,8.41,10.43,5.05,727.70,259.27,54.31,41.38,11.19
6236,692.19,381.08,28.72,28.14,6.98,127.66,75.08,6.92,2.77,0.88,...,108.67,68.99,7.57,8.63,5.47,302.60,159.04,17.64,6.81,1.96
6237,317.38,111.00,21.89,41.00,12.95,39.23,22.45,5.08,9.40,1.13,...,18.51,17.11,6.58,13.99,2.89,374.61,118.01,22.47,23.57,6.42
6238,60.88,14.37,5.74,17.52,8.15,12.85,3.64,1.48,3.94,1.59,...,14.98,8.06,2.87,6.78,4.83,87.70,74.98,32.02,16.02,4.51


### 1. Left vs. Right

In [64]:
X_train, X_eval, y_train, y_eval = train_test_split(eeg_data_five_df.to_numpy(), psd_lr, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4992, 240), (1248, 240), (4992,), (1248,))

In [65]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [00:40<00:00,  1.40s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.83,0.83,0.83,0.83,0.50
RandomForestClassifier,0.83,0.83,0.83,0.83,4.06
XGBClassifier,0.82,0.82,0.82,0.82,1.89
NuSVC,0.80,0.80,0.80,0.80,3.55
SVC,0.80,0.80,0.80,0.80,3.18
CalibratedClassifierCV,0.79,0.79,0.79,0.79,8.57
ExtraTreesClassifier,0.79,0.79,0.79,0.79,0.96
LogisticRegression,0.79,0.79,0.79,0.79,0.09
LinearSVC,0.79,0.79,0.79,0.79,2.08


In [66]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/psd-left-vs-right-five.csv")

### 2. Hand positions

In [67]:
X_train, X_eval, y_train, y_eval = train_test_split(eeg_data_five_df.to_numpy(), psd_pos, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4992, 240), (1248, 240), (4992,), (1248,))

In [68]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [01:09<00:00,  2.41s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.86,0.86,None,0.86,1.77
XGBClassifier,0.86,0.86,None,0.86,5.05
RandomForestClassifier,0.82,0.82,None,0.82,4.55
ExtraTreesClassifier,0.81,0.81,None,0.81,1.02
BaggingClassifier,0.77,0.77,None,0.77,6.96
NuSVC,0.73,0.73,None,0.73,4.84
KNeighborsClassifier,0.68,0.68,None,0.68,0.09
SVC,0.67,0.67,None,0.67,4.51
DecisionTreeClassifier,0.63,0.63,None,0.63,1.19


In [69]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/psd-in-vs-up-vs-down-five.csv")

### 3. Open Vs. Close

In [70]:
X_train, X_eval, y_train, y_eval = train_test_split(eeg_data_all_df.to_numpy(), psd_oc, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((4992, 48), (1248, 48), (4992,), (1248,))

In [71]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_eval, y_train, y_eval)
models

100%|███████████████████████████████████████████| 29/29 [00:18<00:00,  1.58it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.54,0.54,0.54,0.54,0.68
NuSVC,0.53,0.53,0.53,0.53,4.96
NearestCentroid,0.53,0.53,0.53,0.52,0.01
XGBClassifier,0.52,0.52,0.52,0.52,0.62
BaggingClassifier,0.53,0.52,0.52,0.52,1.71
ExtraTreeClassifier,0.52,0.52,0.52,0.52,0.02
LGBMClassifier,0.52,0.52,0.52,0.52,0.18
RidgeClassifier,0.52,0.52,0.52,0.52,0.02
GaussianNB,0.51,0.52,0.52,0.44,0.01


In [72]:
models.to_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results/final/psd-open-vs-close-five.csv")